In [16]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/kprinfo/final_csv.csv')

# Calculate Average Runs in the Last 6 Matches
df['Last1to6MatchRunsAvg'] = df[['RecentMatchRuns1', 'RecentMatchRuns2', 'RecentMatchRuns3', 'RecentMatchRuns4', 'RecentMatchRuns5', 'RecentMatchRuns6']].mean(axis=1)

# Calculate Average Wickets in the Last 6 Matches
df['Last1to6MatchesWicketsAvg'] = df[['RecentMatchWickets1', 'RecentMatchWickets2', 'RecentMatchWickets3', 'RecentMatchWickets4', 'RecentMatchWickets5', 'RecentMatchWickets6']].mean(axis=1)

# Calculate Batting Rating without Batting Average normalization
df['BattingRating(out of 100)'] = (df['Last1to6MatchRunsAvg'] / df['Last1to6MatchRunsAvg'].max()) * 100

# Calculate Bowling Rating without Bowling Average normalization
df['BowlingRating(out of 100)'] = (df['Last1to6MatchesWicketsAvg'] / df['Last1to6MatchesWicketsAvg'].max()) * 100

# Ensure ratings are capped at 100
df['BattingRating(out of 100)'] = df['BattingRating(out of 100)'].clip(upper=100)
df['BowlingRating(out of 100)'] = df['BowlingRating(out of 100)'].clip(upper=100)

# Save the updated dataset to a new CSV file
df.to_csv('/content/drive/MyDrive/kprinfo/with_ratings.csv', index=False)

In [17]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/kprinfo/final_csv.csv')

# Function to handle the player selection
def select_players(team_players):
    selected_players = []
    for player in team_players:
        selected_players.append({'label': player, 'value': player})
    return selected_players

# Dash app with Bootstrap styles
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define layout
app.layout = dbc.Container(
    [
        html.H1("Cricket Team Performance Predictor", className="mt-5 mb-4 text-center"),

        dbc.Row(
            [
                dbc.Col(
                    dcc.Dropdown(
                        id='team-1-dropdown',
                        options=[{'label': team, 'value': team} for team in df['Country'].unique()],
                        multi=False,
                        placeholder="Select Team 1",
                        className="mb-4",
                        style={'font-size': '20px'}
                    ),
                ),
                dbc.Col(
                    dcc.Dropdown(
                        id='team-2-dropdown',
                        options=[{'label': team, 'value': team} for team in df['Country'].unique()],
                        multi=False,
                        placeholder="Select Team 2",
                        className="mb-4",
                        style={'font-size': '20px'}
                    ),
                ),
            ],
            className="mb-4",
        ),

        dbc.Row(
            [
                dbc.Col(
                    dcc.Checklist(
                        id='team-1-players',
                        style={'font-size': '18px'},
                    ),
                    width=6
                ),
                dbc.Col(
                    dcc.Checklist(
                        id='team-2-players',
                        style={'font-size': '18px'},
                    ),
                    width=6
                ),
            ],
            className="mb-4",
        ),

        dbc.Row(
            [
                dbc.Col(
                    dcc.RadioItems(
                        id='performance-option',
                        style={'font-size': '18px'},
                    ),
                    width=6
                ),
                dbc.Col(
                    dcc.Input(
                        id='top-n-input',
                        type='number',
                        value=5,
                        min=1,
                        placeholder="Enter Top N",
                        style={'font-size': '18px'}
                    ),
                    width=6
                ),
            ],
            className="mb-4",
        ),

        dbc.Row(
            [
                dbc.Col(
                    dcc.Dropdown(
                        id='selected-player-dropdown',
                        options=[],
                        multi=False,
                        placeholder="Select Player",
                        className="mb-4",
                        style={'font-size': '18px'}
                    ),
                ),
            ],
            className="mb-4",
        ),

        dbc.Row(
            dbc.Col(html.Div(id='output', style={'font-size': '20px'}), width=12),
            className="mb-5",
        ),

        dbc.Row(
            dbc.Col(
                dbc.Button("Submit", id="submit-button", n_clicks=0, color="primary", className="mb-3"),
                width=12
            ),
            className="mb-5",
        ),
    ],
    fluid=True,
)

# Define callback to update player lists based on selected teams
@app.callback(
    [Output('team-1-players', 'options'),
     Output('team-2-players', 'options'),
     Output('performance-option', 'options'),
     Output('performance-option', 'style'),
     Output('top-n-input', 'style')],
    [Input('team-1-dropdown', 'value'),
     Input('team-2-dropdown', 'value')]
)
def update_player_options(team_1, team_2):
    if team_1 is not None and team_2 is not None:
        team_1_players = select_players(df[df['Country'] == team_1]['Name'].tolist())
        team_2_players = select_players(df[df['Country'] == team_2]['Name'].tolist())

        # Display top performer options only after selecting teams
        performance_options = [
            {'label': 'Overall Performance', 'value': 'overall'},
            {'label': 'Top Batting Performers', 'value': 'batting'},
            {'label': 'Top Bowling Performers', 'value': 'bowling'},
            {'label': 'View Player Performance', 'value': 'view_player'},
        ]

        return team_1_players, team_2_players, performance_options, {'display': 'block'}, {'display': 'block'}
    else:
        return [], [], [], {'display': 'none'}, {'display': 'none'}

# Define callback to update selected player dropdown options
@app.callback(
    Output('selected-player-dropdown', 'options'),
    [Input('team-1-players', 'options'),
     Input('team-2-players', 'options')]
)
def update_selected_player_options(team_1_players, team_2_players):
    return team_1_players + team_2_players

# Define callback to update output based on selected players and options
@app.callback(
    Output('output', 'children'),
    [Input('team-1-players', 'value'),
     Input('team-2-players', 'value'),
     Input('performance-option', 'value'),
     Input('top-n-input', 'value'),
     Input('selected-player-dropdown', 'value')],
    prevent_initial_call=True,  # Skip the initial callback on page load
)
def update_output(selected_players_team_1, selected_players_team_2, performance_option, top_n, selected_player):
    selected_players = selected_players_team_1 + selected_players_team_2
    if len(selected_players) < 22:
        return "Please select at least 11 players from each team."
    else:
        # Filter dataframe for selected players
        selected_players_df = df[df['Name'].isin(selected_players)]

        # Assuming you have the ratings already calculated and saved in a CSV file
        ratings_df = pd.read_csv('/content/drive/MyDrive/kprinfo/with_ratings.csv')

        # Merge the ratings with the original dataset based on the 'Name' column
        selected_players_df = pd.merge(selected_players_df, ratings_df[['Name', 'BattingRating(out of 100)', 'BowlingRating(out of 100)']], on='Name', how='left')

        # Select relevant features and target for the model
        X_columns = ['BattingRating(out of 100)', 'BowlingRating(out of 100)']
        X_selected = selected_players_df[X_columns]

        # Build the Linear Regression models for batting and bowling
        model_batting = LinearRegression()
        model_bowling = LinearRegression()

        # Train the models
        model_batting.fit(X_selected, selected_players_df['BattingRating(out of 100)'])
        model_bowling.fit(X_selected, selected_players_df['BowlingRating(out of 100)'])

        # Predict ratings for selected players
        selected_players_df['PredictedBattingRating'] = model_batting.predict(X_selected)
        selected_players_df['PredictedBowlingRating'] = model_bowling.predict(X_selected)

        # Calculate overall performance based on predicted ratings
        selected_players_df['OverallRating'] = (selected_players_df['PredictedBattingRating'] + selected_players_df['PredictedBowlingRating']) / 2

        if performance_option == 'view_player':
            if selected_player is not None:
                # Display the performance of the selected player
                player_performance = selected_players_df[selected_players_df['Name'] == selected_player]
                player_lines = [f"{name}: {rating}" for name, rating in zip(player_performance['Name'], player_performance['OverallRating'])]
            else:
                player_lines = ["Please select a player to view performance."]
        else:
            if performance_option == 'overall':
                # Display the names and overall performance of top N performers
                top_performers = selected_players_df.nlargest(top_n, 'OverallRating')
            elif performance_option == 'batting':
                # Display the names and batting performance of top N performers
                top_performers = selected_players_df.nlargest(top_n, 'PredictedBattingRating')
            elif performance_option == 'bowling':
                # Display the names and bowling performance of top N performers
                top_performers = selected_players_df.nlargest(top_n, 'PredictedBowlingRating')

            # Create a list of formatted strings for each player
            player_lines = [f"{name}: {rating}" for name, rating in zip(top_performers['Name'], top_performers['OverallRating'])]

        return html.Ul([html.Li(line) for line in player_lines])

# Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [3]:
!pip install pandas
!pip install scikit-learn
!pip install dash
!pip install dash-bootstrap-components


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 3.4 MB/s eta 0:00:00
